<a href="https://colab.research.google.com/github/Idamfuadi/Capstone-Project-Dilaut/blob/ml/Dilaut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dilaut - Bangkit Capstone Project
Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum Lorem ipsum

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

import numpy as np
import pandas as pd 
import time
import math
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#load dataset
data=pd.read_csv('Dilaut Dataset.csv')
data.head()

,Tanggal,Mas,Tawes,Udang Putih/Vaname,Bandeng,Kakap Putih,Mujair,Kerapu,Patin,Gurami,Lele,Udang Windu,Kepiting,Nila
0,01/2016,9000,7500,60000,15000.0,35000,10000,NaN,20000,23500,15000,75000,50000.0,8500
1,02/2016,9500,7500,55000,15000.0,30000,11000,NaN,20000,23500,15500,75000,50000.0,8000
2,03/2016,9500,7500,55000,15000.0,35000,11000,NaN,20000,23500,15500,75000,55000.0,8000
3,04/2016,15000,7500,60000,15000.0,35000,17500,80000.0,17500,23500,14000,75000,50000.0,14000
4,05/2016,15000,7500,60000,15000.0,35000,17500,80000.0,20000,23500,16000,75000,50000.0,14000


In [ ]:
#filling NaN data
median = data.median()
data.fillna(median, inplace=True)
data.head()

,Tanggal,Mas,Tawes,Udang Putih/Vaname,Bandeng,Kakap Putih,Mujair,Kerapu,Patin,Gurami,Lele,Udang Windu,Kepiting,Nila
0,01/2016,9000,7500,60000,15000.0,35000,10000,82000.0,20000,23500,15000,75000,50000.0,8500
1,02/2016,9500,7500,55000,15000.0,30000,11000,82000.0,20000,23500,15500,75000,50000.0,8000
2,03/2016,9500,7500,55000,15000.0,35000,11000,82000.0,20000,23500,15500,75000,55000.0,8000
3,04/2016,15000,7500,60000,15000.0,35000,17500,80000.0,17500,23500,14000,75000,50000.0,14000
4,05/2016,15000,7500,60000,15000.0,35000,17500,80000.0,20000,23500,16000,75000,50000.0,14000


In [ ]:
#get the number of rows to train the model on
df = data.iloc[:, 2:16].values
training_data_len = math.ceil(len(df) * .8)

training_data_len

48

In [ ]:
#scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)
scaled_data

In [ ]:
#create the trainig data set
train_data = scaled_data[0:training_data_len, :]
#split the data into X_train and y_train dataset
X_train = []
y_train = []
'''
for i in range(9, len(train_data)):
  X_train.append(train_data[i-9:i, 0])
  y_train.append(train_data[i, 0])
  if i <= 9:
    print(X_train)
    print(y_train)
    print()
'''

'\nfor i in range(9, len(train_data)):\n  X_train.append(train_data[i-9:i, 0])\n  y_train.append(train_data[i, 0])\n  if i <= 9:\n    print(X_train)\n    print(y_train)\n    print()\n'

In [ ]:
#convert the X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
model = Sequential()
model.add(LSTM(14, activation='relu',input_shape =(1,8)))
model.add(LSTM(20, dropout= 0.8, use_bias= True))
model.add(Dense(13, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='Adam',  metrics=['Precision'])
print(model.summary())